In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
import matplotlib
import pandas as pd
import pickle
from utils.post_processing_utils import *
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [321]:
def moving_average(x, window=50):
    return np.convolve(x, np.ones(window), 'valid') / window

In [399]:
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
state_change_data_file = os.path.join(processed_data_dir, 'state_change_data_nacc_mice.csv')
pre_pc = []
post_pc = []
mice = [ 'SNL_photo21', 'SNL_photo22', 'SNL_photo26']
moving_avs = []
#mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    state_change_data = {}
    date = '20200915' 
    #date = '20201216'
    all_experiments = get_all_experimental_records()
    experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
    contra_side = experiment_to_process['fiber_side'].values[0]
    fiber_options = np.array(['left', 'right'])
    fiber_side_numeric = (np.where(fiber_options != contra_side)[0] + 1)[0]
    if fiber_side_numeric == 2:
        fiber_side_numeric = 7
    print(contra_side,fiber_side_numeric)
    traces, trial_data = open_experiment(experiment_to_process)
    trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
    only_contra = trial_data.loc[trial_data['Trial type'] == fiber_side_numeric] 
    red_trial_data = only_contra[only_contra['State name'] == 'TrialStart']
    post_trials = red_trial_data[np.logical_and(red_trial_data['Trial num'] >= 150, red_trial_data['Trial num'] <= 200)]
    post_pc.append(np.mean(post_trials['Trial outcome'].values) * 100)
    pre_pc.append(np.mean(red_trial_data[red_trial_data['Trial num'] < 150]['Trial outcome'].values) * 100)
    moving_avs.append(moving_average(red_trial_data['Trial outcome'].values, window=20))

left 7
right 1
right 1


In [400]:
num_mice = len(mice)
num_trials = [len(m) for m in moving_avs]
max_num_trials = max([len(m) for m in moving_avs])
all_mice = np.empty((num_mice, max_num_trials))
all_mice[:] = np.nan
for i, mouse_data in enumerate(moving_avs):
    all_mice[i, :num_trials[i]] = mouse_data

In [401]:
fig, axs = plt.subplots()
[axs.plot(m, alpha=0.5, c='gray') for m in moving_avs]
axs.plot(np.nanmean(all_mice,axis=0), c='#7FB5B5', lw=3)

axs.set_xlabel('trial number', fontsize=12)
axs.set_ylabel('performance', fontsize=12)
axs.spines['right'].set_visible(False)
axs.spines['top'].set_visible(False)


In [307]:
[plt.plot(m, alpha=0.5) for m in moving_avs]

[[<matplotlib.lines.Line2D at 0x204bccc0>],

In [246]:
plt.show()

In [211]:
behavioural_change = {}
behavioural_change['mouse'] = mice
behavioural_change['pre performance'] = pre_pc
behavioural_change['post performance'] = post_pc
behavioural_change_df = pd.DataFrame(behavioural_change)

In [212]:
df_for_plot = behavioural_change_df.set_index('mouse').transpose()

In [227]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [226]:
fig, ax = plt.subplots()
francescas_plot(ax, df_for_plot, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['pre state change', 'post state change'], fontsize=12)
plt.ylabel('Performance (%)', fontsize=13)
plt.tight_layout()

In [228]:
df_for_plot

mouse,SNL_photo21,SNL_photo22,SNL_photo26
pre performance,98.000000,95.333333,97.333333
post performance,74.509804,94.117647,72.549020


In [216]:
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
nacc_peaks = pd.read_csv(os.path.join(processed_data_dir, 'nacc_peak_sizes.csv'))
tail_peaks = pd.read_csv(os.path.join(processed_data_dir, 'tail_peak_sizes.csv'))

In [217]:
peaks = tail_peaks 

In [218]:
peak_differences = []
performance_differences = []
for i, row in behavioural_change_df.iterrows():
    mouse = row['mouse']
    peak_differences.append((peaks[mouse][1] - peaks[mouse][0])/peaks[mouse][1]*100)
    performance_differences.append((df_for_plot[mouse][1] - df_for_plot[mouse][0])/df_for_plot[mouse][1]*100)

In [219]:
fig, axs = plt.subplots()
axs.scatter(performance_differences, peak_differences)
axs.set_ylabel('% change in post-pre peak z-score ')
axs.set_xlabel('% difference in performance')


Text(0.5, 0, '% difference in performance')

In [174]:
(peaks[mouse][1] - peaks[mouse][0])

0.5066578094709958

In [175]:
(3-4)/5

-0.2

In [176]:
peaks

,trial type,SNL_photo21,SNL_photo22,SNL_photo26
0,pre,0.266602,0.896028,0.780225
1,post,0.574251,0.894707,1.286883
